# Spam detection using Random Forest

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('Spam_Ham_data.csv')
df

,email,label,Subject,content
0,b'From exmh-workers-admin@redhat.com Thu Aug ...,0.0,new sequenc window,date wed number aug number number number chri ...
1,b'Return-Path: <Online#3.19578.34-UgGTgZFN19NA...,0.0,cnet newscom cabl compani crack wifi,htmlhead titlec compani crack nametopa logo ad...
2,b'Return-Path: <Online#3.19584.83-p1SYlJ1blFvQ...,0.0,save extra number ipaq number pda cnet shopper,htmlhead newslett hreftopa header tabl widthnu...
3,b'From Steve_Burt@cursor-system.com Thu Aug 2...,0.0,zzzzteana alexand,martin greek sculptor behind plan judg limesto...
4,b'Return-Path: <Online#3.19586.b5-9w0blztbvHPd...,0.0,week deck texedit plu boom,cnet download dispatchmac editionjuli number n...
...,...,...,...,...
3058,b'From 2002biz2biz2513@Flashmail.com Mon Oct ...,1.0,fwddirect market work number,stumbl greatest way market centuri undoubtedli...
3059,b'From biz2biz2446@Flashmail.com Mon Oct 7 2...,1.0,see compani sale sky rocket number,stumbl greatest way market centuri undoubtedli...
3060,b'From bounce2@u-answer.com Tue Oct 8 11:02:...,1.0,number hour watch emmerci joke,frontpag numbermeta nameprogid vlinknumb align...
3061,b'From beautyinfufuxxxmeb13mxy@aol.com Tue Oc...,1.0,make fortun ebay number,htmlbodi tr td p number rate work home busi bf...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3063 entries, 0 to 3062
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   email    3063 non-null   object 
 1   label    3063 non-null   float64
 2   Subject  3008 non-null   object 
 3   content  3059 non-null   object 
dtypes: float64(1), object(3)
memory usage: 95.8+ KB


In [4]:
df.head()

,email,label,Subject,content
0,b'From exmh-workers-admin@redhat.com Thu Aug ...,0.0,new sequenc window,date wed number aug number number number chri ...
1,b'Return-Path: <Online#3.19578.34-UgGTgZFN19NA...,0.0,cnet newscom cabl compani crack wifi,htmlhead titlec compani crack nametopa logo ad...
2,b'Return-Path: <Online#3.19584.83-p1SYlJ1blFvQ...,0.0,save extra number ipaq number pda cnet shopper,htmlhead newslett hreftopa header tabl widthnu...
3,b'From Steve_Burt@cursor-system.com Thu Aug 2...,0.0,zzzzteana alexand,martin greek sculptor behind plan judg limesto...
4,b'Return-Path: <Online#3.19586.b5-9w0blztbvHPd...,0.0,week deck texedit plu boom,cnet download dispatchmac editionjuli number n...


In [6]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
label,3063.0,0.131897,0.338434,0.0,0.0,0.0,0.0,1.0


In [9]:
df['label'].value_counts()

0.0    2659
1.0     404
Name: label, dtype: int64

In [10]:
df.isna().sum()

email       0
label       0
Subject    55
content     4
dtype: int64

In [11]:
df.dropna(inplace=True)

In [12]:
df.isna().sum()

email      0
label      0
Subject    0
content    0
dtype: int64

In [13]:
df.columns

Index(['email', 'label', 'Subject', 'content'], dtype='object')

In [14]:
X=df.drop('label',axis=1)
X.head()

,email,Subject,content
0,b'From exmh-workers-admin@redhat.com Thu Aug ...,new sequenc window,date wed number aug number number number chri ...
1,b'Return-Path: <Online#3.19578.34-UgGTgZFN19NA...,cnet newscom cabl compani crack wifi,htmlhead titlec compani crack nametopa logo ad...
2,b'Return-Path: <Online#3.19584.83-p1SYlJ1blFvQ...,save extra number ipaq number pda cnet shopper,htmlhead newslett hreftopa header tabl widthnu...
3,b'From Steve_Burt@cursor-system.com Thu Aug 2...,zzzzteana alexand,martin greek sculptor behind plan judg limesto...
4,b'Return-Path: <Online#3.19586.b5-9w0blztbvHPd...,week deck texedit plu boom,cnet download dispatchmac editionjuli number n...


In [15]:
y=df['label']
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: label, dtype: float64

# Feature Engineering

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [17]:
category=['email','Subject','content']
one_hot_encode=OneHotEncoder()

In [18]:
transformer=ColumnTransformer([("one_hot_encode",one_hot_encode,category)],remainder='passthrough')
X_transformed=transformer.fit_transform(X).toarray()
X_transformed

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
features=pd.DataFrame(X_transformed)
features.head()

,0,1,2,3,4,5,6,7,8,9,...,7448,7449,7450,7451,7452,7453,7454,7455,7456,7457
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Model Building

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(features,y,test_size=0.2,shuffle=True)

In [21]:
X_train.shape,X_test.shape, y_train.shape,y_test.shape

((2403, 7458), (601, 7458), (2403,), (601,))

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
random_forest=RandomForestClassifier(n_estimators=100,
                                     min_samples_split=5,
                                     min_samples_leaf=1,
                                     max_features='auto',max_depth=10).fit(X_train,y_train)

In [24]:
random_forest.score(X_test,y_test)

0.8901830282861897

In [25]:
random_forest_predict=random_forest.predict(X_test)

In [27]:
Y_test=np.array(y_test)
compare_random_forest=pd.DataFrame(data=[Y_test,random_forest_predict])
compare_random_forest=compare_random_forest.transpose()
compare_random_forest.rename(columns={0 :'Real',1:'Predict'},inplace=True)
compare_random_forest

,Real,Predict
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
596,0.0,0.0
597,1.0,0.0
598,1.0,0.0
599,0.0,0.0


In [28]:
from sklearn.model_selection import cross_val_score

In [29]:
cross_random=cross_val_score(random_forest,features,y,cv=10)
np.mean(cross_random)

0.8668438538205981

In [32]:
np.random.seed(50)
for i in range(10,100,10):
    print(f"{i} estimators")
    model=RandomForestClassifier(n_estimators=i).fit(X_train,y_train)
    print(f"Accuracy on test set :{model.score(X_test,y_test)* 100}%")

10 estimators
Accuracy on test set :89.85024958402661%
20 estimators
Accuracy on test set :90.01663893510815%
30 estimators
Accuracy on test set :89.85024958402661%
40 estimators
Accuracy on test set :90.51580698835274%
50 estimators
Accuracy on test set :89.6838602329451%
60 estimators
Accuracy on test set :90.01663893510815%
70 estimators
Accuracy on test set :89.85024958402661%
80 estimators
Accuracy on test set :90.01663893510815%
90 estimators
Accuracy on test set :89.51747088186356%


In [40]:
from sklearn.metrics import confusion_matrix,classification_report

In [38]:
confusion= confusion_matrix(y_test,random_forest_predict)
confusion

array([[535,   0],
       [ 66,   0]], dtype=int64)